In [2]:
from tensorflow import keras
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [3]:
with open('x_test_values.pickle', 'rb') as file:
    x_test_values = pickle.load(file)

with open('y_test.pickle', 'rb') as file:
    y_test = pickle.load(file)

# print(x_test_values[:2])
# print(y_test[:2])

In [4]:
model = keras.models.load_model('model.keras', compile=False)

In [5]:
print(model.summary())
# model.layers[0].mask_value = tf.convert_to_tensor(np.array([0.0, 0.0, 0.0, 0.0]))
# print(model.layers[0].mask_value)
# model.compile()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 352, 4)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 352, 50)        │         8,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 352, 50)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 50)             │        15,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,751 (92.78 KB)

 Trainable params: 23,751 (92.78 KB)

 Non-trainable params: 0 (0.00 B)

None


In [6]:
print(model.layers[0].input.dtype)
print(np.array(x_test_values[:1]).shape)
print(model.input_shape)
y_pred = model.predict(np.array(x_test_values[:1], dtype=np.float32))

float32
(1, 352, 4)
(None, 352, 4)


TypeError: Exception encountered when calling NotEqual.call().

[1mFailed to convert elements of {'class_name': '__tensor__', 'config': {'dtype': 'float64', 'value': [0.0, 0.0, 0.0, 0.0]}} to Tensor. Consider casting elements to a supported type. See https://www.tensorflow.org/api_docs/python/tf/dtypes for supported TF dtypes.[0m

Arguments received by NotEqual.call():
  • x1=tf.Tensor(shape=(1, 352, 4), dtype=float32)
  • x2={'class_name': "'__tensor__'", 'config': {'dtype': "'float64'", 'value': ['0.0', '0.0', '0.0', '0.0']}}

In [ ]:
dftrain = pd.read_csv("data/plasticc_train_metadata.csv")

dftest = pd.read_csv("data/plasticc_test_metadata.csv")

dftest = dftest[:20000]

dftest['target'] = dftest['true_target']

dftrain = pd.concat([dftrain, dftest], axis=0, ignore_index=True)

sn_models = [90, 67, 52, 42, 62, 95, 15]
dftrain = dftrain[dftrain['target'].isin(sn_models)]


In [ ]:
lcs = pd.read_csv("data/plasticc_train_lightcurves.csv")

lcs_test = pd.read_csv("data/plasticc_test_lightcurves_01.csv")

lcs = pd.concat([lcs, lcs_test[:3000000]], axis=0, ignore_index=True)

In [ ]:
def display_object(object_id, lightcurve_data, metadata, predicted=0):
    lightcurve_data = lightcurve_data[lightcurve_data['object_id'] == object_id]
    passbands = []
    true_peaktime = metadata[metadata['object_id'] == object_id]['true_peakmjd']
    print(true_peaktime)

    for freq in range(6):
        passbands.append(lightcurve_data[lightcurve_data['passband'] == freq][['mjd', 'flux']])

    for p in passbands:
        plt.scatter(p['mjd'], p['flux'])
        print(true_peaktime)
        plt.scatter(true_peaktime, [0])
        if predicted > 0:
            plt.scatter([predicted], [0])
        

    # plt.xlabel("Modified Julien Date")

    # plt.ylabel("Flux")
    
    # plt.show()
    plt.show()


In [ ]:
for i in range(3):
    display_object(y_test[i][0], lcs, dftrain, y_pred[i][0]*2000+59000)